# Deep Reinforcement Learning: CartPole with DQN

In [6]:
import gym
import numpy as np
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report


In [7]:
gym.make('CartPole-v1')

<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>

In [8]:
class DQN(nn.Module):   
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        return self.out(x)  

### ReplayBuffer

In [9]:
class ReplayBuffer:
    """A simple replay buffer to store experiences"""
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    # args is a tuple of (state, action, reward, next_state, done)
    def push(self, *args):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = args
        self.position = (self.position + 1) % self.capacity

    # sample a batch of experiences from the buffer
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

### Target Network Update

In [ ]:
def update_target_model(self, soft=False, tau=0.1):
    """
    Update the target model with the online model's weights (main network).
    Parameters:
    - soft: If True, use soft update (tau) instead of hard update.
    - tau: The interpolation factor for soft update.
      """
    

    online_weights = self.model.get_weights()
    target_weights = self.target_model.get_weights()

    if soft:
        new_weights = [
            tau * online + (1 - tau) * target
            for online, target in zip(online_weights, target_weights)
        ]
        self.target_model.set_weights(new_weights)
    else:
        self.target_model.set_weights(online_weights)
    
    

## E-Greedy

In [11]:
def greedy(state, model, epsilon, action_dim):
    if random.random() < epsilon:
        return random.randint(0, action_dim - 1)
    else:
        state = torch.FloatTensor(state).unsqueeze(0)  
        with torch.no_grad():
            q_values = model(state)
        return q_values.argmax().item()


### Target network update mechanism.

In [12]:
def update_target(model, target_model):
    target_model.load_state_dict(model.state_dict())


###  DQN loss function and training loop

In [13]:
def train_step(model, target_model, replay_buffer, optimizer, batch_size, gamma):
    if len(replay_buffer) < batch_size:
        return
    
    

##### Main


In [14]:
model = DQN(input_size=..., hidden_size=..., output_size=...)           
state = [...., ...., ..., ...]
action = greedy(state, model, epsilon=0.1, action_dim=2)


SyntaxError: invalid syntax (270573528.py, line 2)